In [1]:
import torch
from torch.nn import Module
from torch import nn


import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib as plt
import random
import time
import csv
import datetime
import os

In [2]:
# class IMDB_Dataset(T.utils.data.Dataset):
#   # each line: 20 token IDs, 0 or 1 label. space delimited
#   def __init__(self, src_file):
#     all_xy = np.loadtxt(src_file, usecols=range(0,21),
#       delimiter=" ", comments="#", dtype=np.int64)
#     tmp_x = all_xy[:,0:20]   # cols [0,20) = [0,19]
#     tmp_y = all_xy[:,20]     # all rows, just col 20
#     self.x_data = T.tensor(tmp_x, dtype=T.int64) 
#     self.y_data = T.tensor(tmp_y, dtype=T.int64)  # CE loss

#   def __len__(self):
#     return len(self.x_data)

#   def __getitem__(self, idx):
#     token_ids = self.x_data[idx]
#     trgts = self.y_data[idx] 
#     return (token_ids, trgts)

In [3]:
# from torchtext.legacy import datasets

# train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [2]:
test = np.array([[[1, 2, 3], [1, 2, 3], [1, 2, 3]]])

In [6]:
test#.squeeze()


array([[[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]]])

In [2]:
print(*torch.__config__.show().split("\n"), sep="\n")

PyTorch built with:
  - C++ Version: 199711
  - MSVC 192829337
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_37,code=compute_37
  - CuDNN 8.2
  - Magma 2.5.4
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.3, CUDNN_VERSION=8.2.0, CXX_COMPILER=C:/cb/pytorch_1000000000000/work/tmp_bin/sccache-cl.exe, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /w /bigobj -DUSE_PTHREADPOOL -ope

In [2]:
#We try to minimize the randomness as much as possible by setting the random seeds
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)

In [3]:
import arguments

In [4]:
#Loading the set of configurations from the file created by the RunExperiments script
i = arguments.i
training_size = arguments.training_size
batch_size = arguments.batch_size
n_epochs = arguments.n_epochs
learning_rate = arguments.learning_rate
data_type = arguments.data_type
device = arguments.device
weight_initialization = arguments.weight_initialization
framework = arguments.framework
dropout = arguments.dropout
phase = arguments.phase

In [5]:
experiment = 'lstm_{}{}_{}ts_{}batch_{}epochs_{}lr_{}dtype_{}_{}wi_{}dp'.format(framework, i, training_size,
                                                                                batch_size, n_epochs,
                                                                                learning_rate, data_type, device,
                                                                                weight_initialization, dropout)

In [6]:
#making sure tensorflow/keras do not reserve all the VRAM available on GPU
if device == 'gpu':

    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
def load_and_preprocess_data(training_size, vocab_size=10000, review_length=500):
    """This function loads and preprocesses (i.e., pads) the IMDB movie review dataset. The
    function returns the training dataset (according to the passed training size),
    the testing dataset and the larger testing dataset (i.e., the dataset that will be used during
    the inference phase).
    
    
    Arguments
    ---------
    training_size: float
        The proportion of the original training dataset that is used during the training process.
    vocab_size: int
        The number of words that are to be considered among the words that used most frequently.
    review_lenght: int
        The maximum lenght of the movie reviews loaded.
    """
    #Loading the IMDB movie review dataset
    (X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=vocab_size)

    #Setting the training set to correspond to the training size in the configurations
    X_train = X_train[:int(training_size*X_train.shape[0])]
    y_train = y_train[:X_train.shape[0]]

    #Setting up the larger dataset that will be used during the inference phase.
    X_test_ext = X_test.copy()
    y_test_ext = y_test.copy()
    for j in range(8):
        X_test_ext = np.append(X_test_ext, X_test.copy(), axis=0)
        y_test_ext = np.append(y_test_ext, y_test.copy(), axis=0)

    #Padding the reviews so they are all the same lenght
    X_train_padded = keras.preprocessing.sequence.pad_sequences(X_train, maxlen = review_length)
    X_test_padded = keras.preprocessing.sequence.pad_sequences(X_test, maxlen = review_length)
    X_test_padded_ext = keras.preprocessing.sequence.pad_sequences(X_test_ext, maxlen = review_length)
    
    return X_train_padded, y_train, X_test_padded, y_test, X_test_padded_ext, y_test_ext

In [8]:
#Defining dictionaries to set the device and weight initialization configurations
#to their corresponding values in the DL frameworks
device_dict = {
    'cpu': {
        'PyTorch': 'cpu',
        'Keras': '/cpu:0',
        'TensorFlow': '/CPU:0'
    },
    'gpu': {
        'PyTorch': 'cuda',
        'Keras': '/gpu:0',
        'TensorFlow': '/GPU:0'
    }
}

weight_initialization_dict = { 
    'xavier': {
        'PyTorch': torch.nn.init.xavier_normal_,
        'Keras': tf.keras.initializers.GlorotNormal,
        'TensorFlow': tf.compat.v1.initializers.glorot_normal
    },
    'he': {
        'PyTorch': torch.nn.init.kaiming_normal_,
        'Keras': tf.keras.initializers.HeNormal,
        'TensorFlow': tf.compat.v1.keras.initializers.he_normal
    }
}

In [9]:
#If the LSTM model dirrectory (i.e., the directory where the models are saved) does not exist, we create it.
if not os.path.isdir('./models'):
    os.mkdir('./models')
if not os.path.isdir('./models/lstm'):
    os.mkdir('./models/lstm')
if not os.path.isdir('./models/lstm/{}'.format(experiment)):
    os.mkdir('./models/lstm/{}'.format(experiment))

In [10]:
#The set of fixed hyper-parameters
vocab_size = 10000
review_length = 500
embedding_size = 32
hidden_size = 100

In [11]:
#Loading the IMDB dataset
X_train_padded, y_train, X_test_padded, y_test, X_test_padded_ext, y_test_ext = load_and_preprocess_data(training_size,
                                                                                                         vocab_size=10000,
                                                                                                         review_length=500)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\tarek\anaconda3\envs\dl_testing_env\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\tarek\anaconda3\envs\dl_testing_env\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different leng

In [13]:
print('Training {}'.format(experiment))

training_time = 0
inference_time = 0
accuracy = 0

train_start_timestamp = 0
train_end_timestamp = 0

inference_start_timestamp = 0
inference_end_timestamp = 0



if framework == 'PyTorch':
    import pytorch_lstm
    
    pytorch_train_loader, pytorch_test_loader, pytorch_test_loader_ext = pytorch_lstm.generate_pytorch_dataloader(X_train_padded, X_test_padded,
                                                                                                                  X_test_padded_ext, y_train,
                                                                                                                  y_test, y_test_ext, batch_size,
                                                                                                                  device_dict[device][framework],
                                                                                                                  review_length=500)

    model = pytorch_lstm.PyTorchLSTMMod(weight_initialization_dict[weight_initialization][framework],
                                        vocab_size, embedding_size, hidden_size, dropout)
    model = model.to(device_dict[device][framework])
    
    if phase == 'training':
        from torch.optim import Adam
        
        optimizer = Adam(model.parameters(), lr=learning_rate)
        training_time, inference_time, accuracy, train_start_timestamp, train_end_timestamp = pytorch_lstm.pytorch_training_phase(model, optimizer,
                                                                                                                                  pytorch_train_loader, pytorch_test_loader,
                                                                                                                                  n_epochs, device_dict[device][framework],
                                                                                                                                  data_type, experiment)
    elif phase == 'inference':
        inference_start_timestamp, inference_end_timestamp = pytorch_lstm.pytorch_inference_phase(model, experiment, pytorch_test_loader_ext,
                                                                                                  device_dict[device][framework], data_type)
    
    #We take the mean time the model takes to infer a single sample.
    inference_time /= X_test_padded.shape[0]


if framework == 'Keras':
    os.environ['TF2_BEHAVIOR'] = '1'
    import tensorflow as tf

    if device == 'gpu':
        gpus = tf.config.experimental.list_physical_devices('GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
    tf.random.set_seed(0)
    
    import keras_lstm
    
#     if data_type == 'mixed':
#         policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
#         tf.keras.mixed_precision.experimental.set_policy(policy)
    
    if phase == 'training':
    
        model = keras_lstm.initialize_keras_lstm(weight_initialization_dict[weight_initialization][framework],
                                                 vocab_size, review_length, embedding_size, hidden_size,
                                                 dropout)
    
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        loss_fn = tf.keras.losses.binary_crossentropy
        
        training_time, inference_time, accuracy, train_start_timestamp, train_end_timestamp = keras_lstm.keras_training_phase(model, optimizer,
                                                                                                                              loss_fn, X_train_padded,
                                                                                                                              y_train, X_test_padded,
                                                                                                                              y_test, batch_size, n_epochs,
                                                                                                                              device_dict[device][framework],
                                                                                                                              data_type, experiment)
    elif phase == 'inference':
        inference_start_timestamp, inference_end_timestamp = keras_lstm.keras_inference_phase(X_test_padded_ext, y_test_ext,
                                                                                              batch_size, device_dict[device][framework],
                                                                                              data_type, experiment)

if framework == 'TensorFlow':
    #the first version of tensorflow needs to be used, as tensorflow 2.0 uses keras by default
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()

    tf.compat.v1.set_random_seed(0)
    
    import tensorflow_lstm

    
    
    with tf.device(device_dict[device][framework]):
        with tf.compat.v1.variable_scope(name_or_scope='TensorFlowLSTM', reuse=tf.compat.v1.AUTO_REUSE,
                                         initializer=weight_initialization_dict[weight_initialization][framework]):

            model = tensorflow_lstm.TensorFlowLSTMMod(weight_initialization_dict[weight_initialization][framework], vocab_size,
                                                      embedding_size, hidden_size, dropout, device_dict[device][framework])
            
            #Collecting the lenghts of the sequences (note that all the sequences are of the same length as they
            #have been padded).
            lens_train = np.array([len(xi) for xi in X_train_padded], dtype='int32')
            lens_test = np.array([len(xi) for xi in X_test_padded], dtype='int32')
            lens_test_ext = np.array([len(xi) for xi in X_test_padded_ext], dtype='int32')
            if phase == 'training':
            
                training_time, inference_time, accuracy, train_start_timestamp, train_end_timestamp = tensorflow_lstm.tensorflow_training_phase(model, learning_rate,
                                                                                                                                                review_length,
                                                                                                                                                X_train_padded,
                                                                                                                                                lens_train, y_train,
                                                                                                                                                X_test_padded, 
                                                                                                                                                lens_test, y_test, 
                                                                                                                                                batch_size, n_epochs,
                                                                                                                                                device_dict[device][framework],
                                                                                                                                                data_type, experiment)
            elif phase == 'inference':
                
                inference_start_timestamp, inference_end_timestamp = tensorflow_lstm.tensorflow_inference_phase(model, review_length,
                                                                                                                X_test_padded_ext,
                                                                                                                lens_test_ext, y_test_ext,
                                                                                                                batch_size,
                                                                                                                device_dict[device][framework],
                                                                                                                data_type, experiment)

                
#Writing the results collected during the training or the inference phase
if phase == 'training':
    results = {
        'training_time': training_time,
        'inference_time': inference_time,
        'accuracy': accuracy,
        'train_start_timestamp': train_start_timestamp,
        'train_end_timestamp': train_end_timestamp
    }

    # with open('./Results/lstm/{}.txt'.format(experiment), 'w+', encoding='utf-8') as f:
    #     for fieldName in results.keys():
    #         f.write('{} = {}\n\n'.format(fieldName, results[fieldName]))
elif phase == 'inference':
    results = {
        'inference_start_timestamp': inference_start_timestamp,
        'inference_end_timestamp': inference_end_timestamp
    }
    
    # with open('./Results/lstm/{}.txt'.format(experiment), 'a', encoding='utf-8') as f:
    #     for fieldName in results.keys():
    #         f.write('{} = {}\n\n'.format(fieldName, results[fieldName]))

print(results)

Training lstm_Keras0_0.5ts_256batch_3epochs_0.005lr_mixeddtype_gpu_xavierwi_0dp
879/879 [==============================] - 14s 14ms/step - loss: 0.5632 - accuracy: 0.7108
Accuracy: 0.7107599973678589



In [35]:
import time
import csv
import datetime

In [ ]:
def read_results(is_complete=True):
    """This function reads the file that contains the results of the evaluation of the model.
    If this function is called after the inference phase is done (i.e., is_complete=True), then
    the function returns a dictionary containing the training time, the inference time, the accuracy,
    the training and the inference time stamps and the hardware utilization metrics. On the other hand,
    if the function is called before the inference phase (i.e., is_complete=False), then the function
    will return a dictionary containing all the above except for the hardware utilization metrics.
    
    Arguments:
    ----------
    is_complete: Boolean
        This boolean indicates whether the inference phase is done or not.
    """
    results = {}
    with open('./Results/lstm/{}.txt'.format(experiment), 'r', encoding='utf-8') as f:
        
        s = f.read()

        results['training_time'] = float(s.split('training_time = ')[1].split('\n\n')[0])
        results['inference_time'] = float(s.split('inference_time = ')[1].split('\n\n')[0])
        results['accuracy'] = float(s.split('accuracy = ')[1].split('\n\n')[0])

        try:
            results['train_start_timestamp'] = datetime.datetime.strptime(s.split('train_start_timestamp = ')[1].split('\n\n')[0],
                                                                          '%Y-%m-%d %H:%M:%S.%f')
        except:
            results['train_start_timestamp'] = datetime.datetime.strptime(s.split('train_start_timestamp = ')[1].split('\n\n')[0],
                                                                          '%Y-%m-%d %H:%M:%S')

        try:
            results['train_end_timestamp'] = datetime.datetime.strptime(s.split('train_end_timestamp = ')[1].split('\n\n')[0],
                                                                        '%Y-%m-%d %H:%M:%S.%f')
        except:
            results['train_end_timestamp'] = datetime.datetime.strptime(s.split('train_end_timestamp = ')[1].split('\n\n')[0],
                                                                        '%Y-%m-%d %H:%M:%S')



        try:
            results['inference_start_timestamp'] = datetime.datetime.strptime(s.split('inference_start_timestamp = ')[1].split('\n\n')[0],
                                                                              '%Y-%m-%d %H:%M:%S.%f')
        except:
            results['inference_start_timestamp'] = datetime.datetime.strptime(s.split('inference_start_timestamp = ')[1].split('\n\n')[0],
                                                                              '%Y-%m-%d %H:%M:%S')

        try:
            results['inference_end_timestamp'] = datetime.datetime.strptime(s.split('inference_end_timestamp = ')[1].split('\n\n')[0],
                                                                            '%Y-%m-%d %H:%M:%S.%f')
        except:
            new_res['inference_end_timestamp'] = datetime.datetime.strptime(s.split('inference_end_timestamp = ')[1].split('\n\n')[0],
                                                                            '%Y-%m-%d %H:%M:%S')

            
        if is_complete:
            results['cpu_utilization_train'] = [float(samp) for samp in s.split('cpu_utilization_train = [')[1].split('\n\n')[0].replace(']', '').split(', ')]
            results['cpu_mem_train'] = [float(samp) for samp in s.split('cpu_mem_train = [')[1].split('\n\n')[0].replace(']', '').split(', ')]
            results['gpu_utilization_train'] = [float(samp) for samp in s.split('gpu_utilization_train = [')[1].split('\n\n')[0].replace(']', '').split(', ')]
            results['gpu_mem_train'] = [float(samp) for samp in s.split('gpu_mem_train = [')[1].split('\n\n')[0].replace(']', '').split(', ')]

            results['cpu_utilization_infer'] = [float(samp) for samp in s.split('cpu_utilization_infer = [')[1].split('\n\n')[0].replace(']', '').split(', ')]
            results['cpu_mem_infer'] = [float(samp) for samp in s.split('cpu_mem_infer = [')[1].split('\n\n')[0].replace(']', '').split(', ')]
            results['gpu_utilization_infer'] = [float(samp) for samp in s.split('gpu_utilization_infer = [')[1].split('\n\n')[0].replace(']', '').split(', ')]
            results['gpu_mem_infer'] = [float(samp) for samp in s.split('gpu_mem_infer = [')[1].split('\n\n')[0].replace(']', '').split(', ')]
        
    return results

In [ ]:
def collect_sampled_metrics():
    """This function collects the hardware utilization metrics that corresond to the training and inference
    processes according to the time stamps of the training and inference (which are collected from the results
    file) and the time stamps at which the corresponding metric was sampled.
    """
    results = read_results(is_complete=False)

    results['cpu_utilization_train'] = []
    results['cpu_mem_train'] = []
    results['gpu_utilization_train'] = []
    results['gpu_mem_train'] = []
    
    results['cpu_utilization_infer'] = []
    results['cpu_mem_infer'] = []
    results['gpu_utilization_infer'] = []
    results['gpu_mem_infer'] = []


    with open('./Results/lstm/metric_sampling.csv', 'r', encoding='utf-8') as csvFile:
        csvReader = csv.reader(csvFile, delimiter=',')

        for row in csvReader:
            if row[0] == 'CPU Utilization':
                continue

            sample_timestamp_str = row[4]

            try:
                sample_timestamp_datetime = datetime.datetime.strptime(sample_timestamp_str, '%Y-%m-%d %H:%M:%S.%f')
            except:
                sample_timestamp_datetime = datetime.datetime.strptime(sample_timestamp_str, '%Y-%m-%d %H:%M:%S')


            if results['inference_end_timestamp'] < sample_timestamp_datetime:
                break
            
            if results['inference_start_timestamp'] <= sample_timestamp_datetime and results['inference_end_timestamp'] >= sample_timestamp_datetime:
                results['cpu_utilization_infer'].append(float(row[0]))
                results['cpu_mem_infer'].append(float(row[1]))

                results['gpu_utilization_infer'].append(float(row[2]))
                results['gpu_mem_infer'].append(float(row[3]))
                continue

            if results['train_start_timestamp'] > sample_timestamp_datetime:
                continue

            if results['train_end_timestamp'] < sample_timestamp_datetime:
                continue

            
                
            results['cpu_utilization_train'].append(float(row[0]))
            results['cpu_mem_train'].append(float(row[1]))

            results['gpu_utilization_train'].append(float(row[2]))
            results['gpu_mem_train'].append(float(row[3]))


    with open('./Results/lstm/{}.txt'.format(experiment), 'w+', encoding='utf-8') as f:
        for fieldName in results.keys():
            f.write('{} = {}\n\n'.format(fieldName, results[fieldName]))

In [ ]:
#We only collect the hardware utilization metrics after the inference phase
if phase == 'inference':
    collect_sampled_metrics()